In [1]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import torch
import torch.nn.functional as F
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
import os

### Set up

In [2]:
device = str(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
print(device)

cuda


In [3]:
from huggingface_hub import HfFolder, whoami
load_dotenv()
hf_token = os.getenv('HF_TOKEN')
HfFolder.save_token(hf_token)
user = whoami()
print(f"logged in as {user["name"]}")

logged in as M00nl8tshad0w


In [4]:
def load_model(model_name, local_dir="./models/llama3_70b"):
    if os.path.exists(local_dir):
        print(f"Loading model from local directory: {local_dir}")
        tokenizer = AutoTokenizer.from_pretrained(local_dir)
        model = AutoModelForCausalLM.from_pretrained(local_dir, device_map="auto", torch_dtype="auto")
    else:
        print(f"Local directory not found. Downloading model '{model_name}' from Hugging Face Hub...")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto")

        os.makedirs(local_dir, exist_ok=True)
        tokenizer.save_pretrained(local_dir)
        model.save_pretrained(local_dir)
        print(f"Model downloaded and saved locally to: {local_dir}")

    return tokenizer, model

#tokenizer, model = load_model(model_name="meta-llama/Llama-3.3-70B-Instruct",
#                              local_dir="/home/max/Studium/Leipzig/Semster6/Math_and_ML/hf_models/llama3_70b/")

tokenizer, model = load_model(model_name="THUDM/GLM-4-Z1-9B-0414",
                              local_dir="/home/max/Studium/Leipzig/Semster6/Math_and_ML/hf_models/GLM-4-Z1-9B-0414")

Loading model from local directory: /home/max/Studium/Leipzig/Semster6/Math_and_ML/hf_models/GLM-4-Z1-9B-0414


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [5]:
def load_hf_dataset(dataset_name, subset="default", local_dir="~/hf_datasets/OpenR1_Math_220k/"):
    local_dir = os.path.expanduser(local_dir)
    os.makedirs(local_dir, exist_ok=True)
    return load_dataset(dataset_name, subset, cache_dir=local_dir)

math_dataset = load_hf_dataset(dataset_name="open-r1/OpenR1-Math-220k",
                               local_dir="/home/max/Studium/Leipzig/Semster6/Math_and_ML/hf_datasets/open-r1/OpenR1-Math-220k")

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

### Generation

In [5]:
tokenizer.pad_token_id = tokenizer.eos_token_id
inputs = tokenizer(["What is 1+1? Give a short answer."], return_tensors="pt")
inputs = inputs.to(device)

In [10]:
# Example 1: Print the scores for each token generated with Greedy Search
outputs = model.generate(**inputs, max_new_tokens=50, return_dict_in_generate=True, output_scores=True)
print(f"Generated: {outputs}")
transition_scores = model.compute_transition_scores(
    outputs.sequences, outputs.scores, normalize_logits=True
)

Generated: GenerateDecoderOnlyOutput(sequences=tensor([[ 3838,   374,   220,    16,    10,    16,    30, 20635,   264,  2805,
          4226,    13, 13699,   766,   397, 32169,    11,   773,   279,  1196,
           374, 10156,    11,   330,  3838,   374,   220,    16,    10,    16,
          7521,   323,  6801,   264,  2805,  4226,    13,  6771,   752,  1744,
            13,  8324,    11,   220,    16,  5519,   220,    16,   374,  5020,
         30222,    13,   758,  6770, 34612,    11,  7842,   220,    16,   323,
           220,    16]], device='cuda:0'), scores=(tensor([[-1.8047, -2.4531, -1.5234,  ..., -1.5781, -1.5781, -1.5625]],
       device='cuda:0'), tensor([[ 3.7031,  3.0156,  5.6875,  ..., -6.4375, -6.4375, -6.4375]],
       device='cuda:0'), tensor([[ 5.5000, -2.2188, -1.2422,  ...,  0.9062,  0.9062,  0.9062]],
       device='cuda:0'), tensor([[-0.3926,  2.1875,  3.3906,  ..., -2.6406, -2.6406, -2.6406]],
       device='cuda:0'), tensor([[ 7.5625, -2.2969, -4.0625,  ..., -2

In [11]:
input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]
print("| token | token string | log probability | probability")
for tok, score in zip(generated_tokens[0], transition_scores[0]):
    # | token | token string | log probability | probability
    print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.detach().cpu().numpy():.3f} | {np.exp(score.detach().cpu().numpy()):.2%}")

| token | token string | log probability | probability
| 13699 | <th      | -0.170 | 84.36%
|   766 | ink      | 0.000 | 100.00%
|   397 | >
       | -0.000 | 100.00%
| 32169 | Okay     | -0.002 | 99.76%
|    11 | ,        | -0.000 | 100.00%
|   773 |  so      | -0.421 | 65.65%
|   279 |  the     | -0.226 | 79.74%
|  1196 |  user    | -0.430 | 65.07%
|   374 |  is      | -0.181 | 83.44%
| 10156 |  asking  | -0.000 | 100.00%
|    11 | ,        | -0.657 | 51.82%
|   330 |  "       | -0.000 | 99.99%
|  3838 | What     | -0.000 | 100.00%
|   374 |  is      | -0.000 | 100.00%
|   220 |          | -0.000 | 100.00%
|    16 | 1        | -0.000 | 100.00%
|    10 | +        | -0.002 | 99.84%
|    16 | 1        | -0.000 | 100.00%
|  7521 | ?"       | -0.100 | 90.46%
|   323 |  and     | -0.073 | 92.95%
|  6801 |  wants   | -0.349 | 70.56%
|   264 |  a       | -0.000 | 100.00%
|  2805 |  short   | -0.000 | 99.98%
|  4226 |  answer  | -0.000 | 100.00%
|    13 | .        | -0.001 | 99.94%
|  6771 | 

### Logits

In [12]:
logits = outputs.scores  # This is a list of logits for each token generated
print(f"{len(logits)=}")
print(f"{logits[0].shape=}")
print(f"{logits[0]=}")
probabilities = [torch.nn.functional.softmax(logit, dim=-1) for logit in logits] # Convert logits to probabilities using softmax
print(f"{probabilities[0].shape=}")
print(f"{probabilities[0]=}")

len(logits)=50
logits[0].shape=torch.Size([1, 151552])
logits[0]=tensor([[-1.8047, -2.4531, -1.5234,  ..., -1.5781, -1.5781, -1.5625]],
       device='cuda:0')
probabilities[0].shape=torch.Size([1, 151552])
probabilities[0]=tensor([[3.4848e-09, 1.8220e-09, 4.6165e-09,  ..., 4.3709e-09, 4.3709e-09,
         4.4397e-09]], device='cuda:0')


In [36]:
probabilities[0].squeeze(0)[0:10]

tensor([3.4848e-09, 1.8220e-09, 4.6165e-09, 8.1658e-09, 1.4641e-09, 5.8816e-09,
        1.3385e-08, 2.2071e-07, 1.3280e-08, 1.3330e-09], device='cuda:0')

In [43]:
def get_token_distributions(probabilites: list) -> list:
    distributions = []
    for token in probabilites:
        token_probabilities = {}
        token_tensor = token.squeeze(0)
        """prob_values = torch.isclose(token_tensor, torch.tensor(float(0.0)))
        for i, bool in enumerate(prob_values):
            if not bool:"""
        threshold = 1e-3
        for i, prob in enumerate(token_tensor):
            if prob.item() > threshold:
                actual_token = tokenizer.decode(i)
                token_prob = token_tensor[i]
                print(f"Probability of '{actual_token}':", end=' ')
                print(f"{token_prob.item():.4%}")
                token_probabilities[actual_token] = token_prob
        print("----")
        distributions.append(token_probabilities)
    return distributions

In [44]:
distributions = get_token_distributions(probabilities)

Probability of ' ': 3.2707%
Probability of ' (': 0.2522%
Probability of ' A': 0.1051%
Probability of ' -': 0.1191%
Probability of ' The': 1.0618%
Probability of ' 
': 0.2685%
Probability of ' In': 0.2226%
Probability of ' It': 0.2685%
Probability of ' If': 0.9975%
Probability of '  
': 0.1350%
Probability of ' 

': 3.7061%
Probability of ' Then': 0.3447%
Probability of ' Also': 0.9371%
Probability of '<th': 84.3512%
Probability of ' Explain': 0.4426%
----
Probability of 'ink': 100.0000%
----
Probability of '>
': 99.9998%
----
Probability of 'Okay': 99.7585%
Probability of 'Alright': 0.1926%
----
Probability of ',': 99.9958%
----
Probability of ' the': 24.1514%
Probability of ' so': 65.6504%
Probability of ' let': 10.0678%
----
Probability of ' the': 79.7350%
Probability of ' I': 20.1602%
----
Probability of ' user': 65.0680%
Probability of ' question': 34.8284%
----
Probability of ' is': 83.4444%
Probability of ' asked': 16.4312%
----
Probability of ' asking': 99.9996%
----
Probability

### Entropy + Plotting

In [47]:
def plot_token_probabilities(token_prob_dict, path, filename):    
    sorted_tokens = token_prob_dict.keys()
    sorted_tokens = ["_" if token==" " else token.replace(" ", "_") for token in sorted_tokens]
    probabilities = [p.item() for p in token_prob_dict.values()]
    
    entropy = -sum(p * np.log(p) for p in probabilities if p > 0) #show entropy too
    normalized_entropy = entropy / np.log(len(probabilities))
    
    # Create a figure and axis
    plt.figure(figsize=(12, 6))

    # Create the bar plot
    plt.bar(sorted_tokens, probabilities, color='lightblue', edgecolor='black', linewidth=0.5)

    # Set axis labels and title
    plt.xlabel('token', fontsize=14, family='sans-serif')
    plt.ylabel('P(token)', fontsize=14, family='sans-serif')
    plt.title(f'Token Probabilities\nNormalized Entropy: {normalized_entropy:.3f}', fontsize=16, family='sans-serif', fontweight='bold')

    # Customize ticks and axes
    plt.xticks(rotation=90, ha='right', fontsize=12)
    plt.yticks(fontsize=12)
    plt.gca().set_facecolor('white')
    plt.grid(False)

    # Clean up spines
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(0.5)
    ax.spines['bottom'].set_linewidth(0.5)

    # Tight layout for better spacing
    plt.tight_layout()

    os.makedirs(path, exist_ok=True)
    full_path = os.path.join("plots", filename)
    plt.savefig(full_path)
    plt.close()
    print(f"Plot saved to {full_path}")

In [48]:
c = 0
stop = 10
for i,token_distribution in enumerate(distributions):
    plot_token_probabilities(token_distribution, "plots", f"token_{i}")
    c += 1
    if c == stop:
        break

Plot saved to plots/token_0
Plot saved to plots/token_1
Plot saved to plots/token_2
Plot saved to plots/token_3


/tmp/ipykernel_316/160312386.py:7: RuntimeWarning: invalid value encountered in scalar divide
  normalized_entropy = entropy / np.log(len(probabilities))
/tmp/ipykernel_316/160312386.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  normalized_entropy = entropy / np.log(len(probabilities))


Plot saved to plots/token_4
Plot saved to plots/token_5
Plot saved to plots/token_6
Plot saved to plots/token_7
Plot saved to plots/token_8
Plot saved to plots/token_9


In [54]:
def get_entropies(distributions):
    def calculate_entropy(probs, n):
        entropy = -torch.sum(probs * torch.log(probs + 1e-10), dim=-1).item() 
        normalized_entropy = entropy / np.log(n) if np.log(n) > 0 else 0
        return normalized_entropy
    entropies = []
    for distribution in distributions:
        probabilities = torch.stack(list(distribution.values()))
        print(probabilities)
        print()
        entropies.append(calculate_entropy(probabilities, len(distribution)))
    print(f"Entropy for eachs token distribution: {entropies}")
    return entropies

In [ ]:
entropies = get_entropies(distributions)
print("--check--")
print(entropies)